# 畳み込みニューラルネットワーク (CNN)
MLPとは異なり、畳み込み層（Convolutional Layer）やプーリング層（Pooling Layer）を活用して特徴を抽出する仕組みを持っている  
## CNNの基本構造
### 1.畳み込み層（Convolutional Layer）
局所的な特徴（エッジ・テクスチャ・形状など）を学習。  
カーネル（フィルタ）を使い、スライドしながら入力画像と演算を行う。
### 活性化関数（Activation Function）
ReLUを使い、非線形変換を適用。
### プーリング層（Pooling Layer）
特徴マップをダウンサンプリングし、計算量を削減しながら位置不変性を持たせる。
ダウンサンプリング：データのサイズを縮小する処理  
位置不変性：入力画像の位置が多少変わっても、モデルの出力が大きく変わらない性質のこと  
### 全結合層（Fully Connected Layer）
最終的なクラス分類を行う。
## メリット
局所特徴を捉えられる  
パラメータ数が少ない
画像の位置ずれに強い  
フィルタが自動学習される
## デメリット
計算コストが高い  
テキストデータには向かない  
過学習しやすい

In [1]:
import torch
import torch.nn as nn


#CNN モデル定義
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)  # 畳み込み層1
        self.pool = nn.MaxPool2d(2, 2)  # プーリング層
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)  # 畳み込み層2
        self.fc = nn.Linear(32 * 7 * 7, 10)  # 全結合層

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # フラット化
        x = self.fc(x)
        return x



In [3]:
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

#データの前処理
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset  = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

#モデル定義
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#学習ループ
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    #テスト精度計算
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Epoch {epoch+1}/{num_epochs}, Accuracy: {correct/total:.4f}")


Failed to download (trying next):
HTTP Error 404: Not Found



100%|████████████████████████████| 9.91M/9.91M [00:05<00:00, 1.75MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|█████████████████████████████| 28.9k/28.9k [00:00<00:00, 737kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|████████████████████████████| 1.65M/1.65M [00:00<00:00, 3.09MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|████████████████████████████| 4.54k/4.54k [00:00<00:00, 1.18MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Epoch 1/5, Accuracy: 0.9829
Epoch 2/5, Accuracy: 0.9817
Epoch 3/5, Accuracy: 0.9892
Epoch 4/5, Accuracy: 0.9891
Epoch 5/5, Accuracy: 0.9876
